# Applicative

## Outline

* Incentive for Applicative

* Definition of Applicative type class

* Using Applicative and Functor together

* Applicative examples

* Applicative laws

In this lesson, we will learn about the Applicative type classe and how you can use it.

## Incentive for Applicative

Imagine you have a function `maybeAdd1 :: Num a => Maybe (a -> a)`. This means that the function migh exist or not. 

How can this be? Well a practical example is if you use the `<$>` operator with a function like `(+)` and apply it to a `Maybe a` value.

In [ ]:
maybeAddition :: Num a => Maybe a -> Maybe (a -> a)
maybeAddition maybeVal = (+) <$> maybeVal

-- Here is a more detailed breakdown of this function without using <$>
maybeAddition' :: Num a => Maybe a -> Maybe (a -> a)
maybeAddition' Nothing = Nothing
maybeAddition' (Just n) = Just $ (+) n

var1 :: Maybe Int
var1 = Just 1

maybeAdd1 = maybeAddition var1
:t maybeAdd1

We see that the command `:t` in the code above returns the signature we defined in the begining where type `a` is now `Int`. 

Now comes the question if you want to use this function on another `Maybe Int` variable how would you do it? 

The function would then have the type signature:
```haskell
add1 :: Maybe Int -> Maybe Int
```
One possibility would be to write the `add1` function such that handles the case of the missing function.

In [ ]:
import Data.Maybe ( fromJust )

var2 :: Maybe Int
var2 = Just 2

add1 :: Maybe Int -> Maybe Int
add1 Nothing = Nothing
add1 (Just n) = Just $ fromJust maybeAdd1 n

print $ add1 var2

Instead of using pattern matching when defining the `add1` function we can also use the Applicative type class to solve this problem.

## Definition of Applicative type class

The Applicative type class is defined as follows:
```haskell
class Functor f => Applicative f where
  pure :: a -> f a
  (<*>) :: f (a -> b) -> f a -> f b
  GHC.Base.liftA2 :: (a -> b -> c) -> f a -> f b -> f c
  (*>) :: f a -> f b -> f b
  (<*) :: f a -> f b -> f a
  {-# MINIMAL pure, ((<*>) | liftA2) #-}
```

We see that Functor is a superclass of Applicative. This means that every type that has an instance of Applicative also has to have an instance of Functor.

The minimal complete definition requires the `pure` operator and another operator which can be the operator `<*>` called app or the function `liftA2`. 

The `pure` function takes a type and puts it into a context. Here is an example with Maybe and lists which both have an instance of Applicative.

var1 = pure 1 :: Maybe Int
var2 = pure 1 :: [Int]

print var1
print var2

Ussually when we work with a type that has an instance of Applicative we are using the `<*>` operator and more rarely the `liftA2` function.

The `<*>` operator takes a function and a variable both in the same context and applies the function to the variable after taking them out of their context. 

Then it puts the result back in the context. Let's see how it can help us to easier defining the `add1` function.

In [ ]:
var1 = 1 :: Int
var2 = Just 1 :: Maybe Int

add1 :: Maybe Int -> Maybe Int
add1 maybeInt = maybeAdd1 <*> maybeInt

print $ add1 (pure var1)
print $ add1 var2

We get the same result as before and the function is one line shorter. A good reason to use the `<*>` operator is that it's easier to read the code because it's generic. 

The same symbol serves the same purpose for all Applicatives. And for types that have already an instance of Applicative as Maybe and lists, the `<*>` operator is already defined.

## Using Applicative and Functor together

If we think a bit about the examples from the previous chapter we may ask ourselves how to write a function `add` that takes in two Maybe values and returns also a Maybe value.

In [ ]:
add :: Num a => Maybe a -> Maybe a -> Maybe a
add maybeVal1 maybeVal2 = (+) <$> maybeVal1 <*> maybeVal2

print $ add var1 var2

We see we get again the same result and used much less lines of code. Because of how `<*>` is defined you can always add one more parameter to the function. 

Also instead of using the `<$>` operator you can use `pure` that puts the function in the begining directly in the maybe context.

In [ ]:
var3 :: Maybe Int
var3 = Just 3

addThree :: Num a => Maybe a -> Maybe a -> Maybe a -> Maybe a
addThree maybeVal1 maybeVal2 maybeVal3 = pure (+) <*> (pure (+) <*> maybeVal1 <*> maybeVal2) <*> maybeVal3

print $ addThree var1 var2 var3

## Applicative examples

Another way how to use the Applicative type class is in IO which has also an instance of Applicative. 

Let's imagine you ask the user for two numbers and then calculate their product. Here is the code where we make use of the `<*>` and `<$>` operators.

In [ ]:
myProduct :: Num a => a -> a -> a
myProduct x1 x2 = x1*x2

readDouble :: IO Double
readDouble = read <$> getLine

ioProduct :: IO Double
ioProduct = myProduct <$> readDouble <*> readDouble

main :: IO ()
main = do
    print "Input 2 numbers. Use dot for decimal seperator."
    result <- ioProduct
    print result

main

You can also use the `<*>` operator on lists to get all possible combinations. 

This is because if you view a list type as a context instead as a container, you can say that a list represents more possible values for a fixed type. 

For instance `[Int]` represent more possible values of type `Int`. Here is a code example:

In [ ]:
list1 = [1,2,3] :: [Int]
list2 = [4,5] :: [Int]

allCombinations :: Num a => [a] -> [a] -> [a]
allCombinations l1 l2 = pure (+) <*> l1 <*> l2

print $ allCombinations list1 list2

Another example for creating all possible combination is by using lists to create an instance of a user defined type with record syntax. Here is the code:

In [ ]:
data Car = Car {
    company :: String,
    color :: String
} deriving Show

companies = ["Toyota", "Mercedes", "Ford"]
colors = ["yellow", "gree", "blue"]

allPossibleCars :: [Car]
allPossibleCars = pure Car <*> companies <*> colors

print allPossibleCars

In our final example let's create an Applicative instance for the Wrapper type we defined in the previous lesson.
```haskell
data Wrapper a = Empty | Wrapper a deriving Show
```

We will first need to define the function that will work as the `<*>` operator. The pure function we can define directly.

In [ ]:
data Wrapper a = Empty | Wrapper a deriving Show

appWrapper :: Wrapper (a -> b) -> Wrapper a -> Wrapper b
appWrapper f Empty = Empty
appWrapper Empty x = Empty
appWrapper (Wrapper f) (Wrapper n) = Wrapper (f n)

instance Functor Wrapper where
    fmap f val = (pure f) <*> val   

instance Applicative Wrapper where
    (<*>) = appWrapper
    pure val = Wrapper val

Now we can define an IO function that askes the user for two numbers, creates Wraper types from them and summs those numbers together.

In [ ]:
import Data.Char (isDigit)

sumWrapperNums :: IO ()
sumWrapperNums = do
    putStrLn "Input first number:"
    n1 <- getLine
    putStrLn "Input second number:"
    n2 <- getLine

    let test = all isDigit n1 && all isDigit n2
    if test 
    then do
        let w1 = pure (read n1) :: Wrapper Int
            w2 = pure (read n2) :: Wrapper Int
        putStrLn "Wrapper sum is:"
        print $ (+) <$> w1 <*> w2
    else do
        putStrLn "The input data should be only digits. Try again."
        sumWrapperNums

sumWrapperNums

## A more complex example

If we check in Haskell what type classes a function in general belongs to, we see it has also an instance of Applicative.
```haskell
:i (->)
type (->) :: * -> * -> *
data (->) a b
infixr -1 ->
instance Applicative ((->) r) 
instance Functor ((->) r) 
instance Monad ((->) r) 
instance Monoid b => Monoid (a -> b) 
instance Semigroup b => Semigroup (a -> b) 
```

The definition of the Applicative instance for a funtion is as follows:
```haskell
instance Applicative ((->) r) where
    pure = const
    (<*>) f g x = f x (g x)
```

The second definition is valid only if `f` is a function that takes in two parameters and `g` is a function that takes in one parameter. 

Here is an example how we can use this to construct a fibonacci list with `<*>` where we also use a recursive call to the list itself.

In [ ]:
fibs = 0 : 1 : (zipWith (+) <*> tail) fibs

print $ take 5 fibs

## Applicative laws

Same as for Functor the Applicative type class has also its origins in mathematics and is defined with laws.

Applicative has 4 laws. They are as follows:

- Identity:<br>`pure id <*> x = x`

- Composition:<br>`pure (.) <*> x1 <*> x2 <*> x3 = x1 <*> (x2 <*> x3)`

- Homomorphism:<br>`pure f <*> pure x = pure (f x)`

- Interchange:<br>`x1 <*> pure x2 = pure ($ x2) <*> x1`

The identity law ensures that `<*>` should only be a mapping function and should not change anything in the applicative except for the value that it’s mapping.

The composition law says that independent of which way we chose to apply our functions, we should always get the same result.

The homomorphism law says that `<*>` should not be doing anything else then applying the mapping.

The interchange law states that we can flip the operands of `<*>` in a predictable way.

Below is example code that proves these laws hold for the Maybe type which has an instance of Applicative.

In [ ]:
-- Identity law
(pure id <*> (Just 1)) == Just 1
(pure id <*> Nothing) == Nothing

-- Composition law
add1, mult2 :: Maybe (Integer -> Integer)
add1 = (+) <$> (Just 1)
mult2 = (*) <$> (Just 2)

(pure (.) <*> add1 <*> mult2 <*> (Just 1)) == (add1 <*> (mult2 <*> (Just 1)))
(pure (.) <*> Nothing <*> mult2 <*> (Just 1)) == (Nothing <*> (mult2 <*> (Just 1)))

-- Homomorphism law
add2 :: Num a => a -> a
add2 x = x + 2
(pure add2 <*> pure 1 :: Maybe Int) == (pure (add2 1) :: Maybe Int)

-- Interchange law
(add1 <*> pure 1) == (pure ($ 1) <*> add1)

Let's look now at an example where we create a type and an Applicative instance for it that violates these laws.

In [ ]:
data NotOk a = NotOk a Bool deriving (Eq, Show)

appNotOk :: NotOk (a -> b) -> NotOk a -> NotOk b
appNotOk (NotOk f myBool1) (NotOk x myBool2) =
    NotOk (f x) (not myBool2)

pureNotOk :: a -> NotOk a 
pureNotOk x = NotOk x True

instance Functor NotOk where
    fmap f x = pure f <*> x

instance Applicative NotOk where
    pure = pureNotOk
    (<*>) = appNotOk

var1 :: NotOk Int
var1 = NotOk 1 True

add1, mult2 :: NotOk (Int -> Int)
add1 = (+) <$> (NotOk 1 True)
mult2 = (*) <$> (NotOk 2 True)

add2 :: Num a => a -> a
add2 x = x + 2

print $ (pure id <*> var1) == var1
print $ (pure (.) <*> add1 <*> mult2 <*> var1) == (add1 <*> (mult2 <*> var1))
print $ (pure add2 <*> pure 1 :: NotOk Int) == (pure (add2 1) :: NotOk Int)
print $ (add1 <*> pure 1) == (pure ($ 1) <*> add1)

Same as for Functor it is not obligatory to follow these laws in Haskell if you create a type and make an instance of Applicative for it.

It is still good practice to follow them because:
- you can better reason about what your code is doing
- you can make use of other functions that work with the Applicative type class

## Recap

In this lesson we've discussed:

- the motivation for introducing Applicative type class 

- definition of the Applicative type class

- how to use the `<*>` operator together with `<$>`

- examples that show how to use the Applicative type class

- laws that apply to the Applicative type class